# Learn v75 med walkthrough-metoden

In [1]:
import pandas as pd 
import numpy as np 
from catboost import CatBoostClassifier,Pool,utils

import sys
sys.path.append('C:\\Users\peter\\Documents\\MyProjects\\PyProj\\Trav\\spel')
import V75_scraping as vs

In [2]:
def scrape_nya_lopp():
    nya_lopp,strukna = vs.v75_scraping(resultat=True,history=True)

    df=pd.concat([pd.read_csv('all_data.csv'), nya_lopp])
    print('shape med nya lopp',df.shape)
    #ta bort dubletter
    df.drop_duplicates(['datum','avd','häst'],inplace=True)
    df.sort_values(by=['datum','avd'],inplace=True)
    df.reset_index(drop=True,inplace=True)
    print('shape med dubletter bort',df.shape)

    df.to_csv('all_data.csv', index=False)

    print("första datum i df =",df.datum.head(1).to_list()[0])
    print("sista  datum i df =",df.datum.tail(1).to_list()[0])

    return df,nya_lopp

In [3]:
### beräkna vilka datum att använda ###
def get_alla_datum(proc=0.75, total_omlärning = False):
    if total_omlärning:
        nya_lopp=None
        df = pd.read_csv('all_data.csv')     
        alla_datum = df.datum.unique()
        split_ix = int(len(alla_datum)*proc)
    else:
        # normalt adderar vi bara 1 eller flera veckor från "omg_att_spela_link.csv"
        df, nya_lopp = scrape_nya_lopp()  # scrape från 'omg_att_spela_link.csv' och addera till df
        omg_df = pd.read_csv('..\\omg_att_spela_link.csv')     
        startix=omg_df.Link.str.find('spel')[0]    # index till 'spel' i url
        alla_datum = omg_df.Link.str.slice(start=startix+5,stop=startix+15).to_list()  
        split_ix=0
        print(f'datum att lära från {alla_datum}')

    return df,nya_lopp,alla_datum,split_ix


In [4]:
### returnera en modell med parametrar satta
def get_model(d=6,l2=2,iterations=3000,use_best=True,verbose=False):
    model = CatBoostClassifier(iterations=iterations,use_best_model=use_best, 
        custom_metric=['Logloss', 'AUC','Recall', 'Precision', 'F1', 'Accuracy'],

        eval_metric='Accuracy', 
        depth=d,l2_leaf_reg=l2,
        auto_class_weights='Balanced',verbose=verbose, random_state=2021) 
    return model                

In [5]:
### Features som inte används vid träning
def remove_features(df,remove_mer=[]):
    #remove_mer=['h5_perf','h5_auto','h4_perf','h4_auto', 'h3_perf', 'h2_perf']
    df.drop(['avd','startnr','vodds','podds','bins','h1_dat','h2_dat','h3_dat','h4_dat','h5_dat'],axis=1,inplace=True) #
    if remove_mer:
        df.drop(remove_mer,axis=1,inplace=True)
    
    # df=check_unique(df.copy())
    # df=check_corr(df.copy())
    return df

In [6]:
 ## byt ut alla NaN till text för cat_features
def replace_NaN(X_train,X_test=None, cat_features=[]):
    # print('cat_features',cat_features)
    for c in cat_features:
        # print(c)
        X_train.loc[X_train[c].isna(),c] = 'None'       ### byt ut None-värden till texten 'None
        if X_test is not None:  ## om X_test är med
            X_test.loc [X_test[c].isna(),c] = 'None'    ### byt ut None-värden till texten 'None

    return X_train,X_test

## Walkthrough startar här

In [7]:
### Kör en walkthrough learn här, en datum i taget framåt

# Jag har ändrat till att alla steg kör utan test-datam ed fast iterations=100
def walkthrough(classic_test=False, verbose=False):
    
    df, nya_lopp, alla_datum, split_ix = get_alla_datum()

    l2_leaf_regs=2
    model=get_model(use_best=False,iterations=100)
    df=remove_features(df.copy())
    cat_features = list(df.loc[:,df.dtypes=='O'].columns)
    df,_ = replace_NaN(df.copy(), cat_features=cat_features)    
    print(f'cat_features {cat_features}\n')

    df['plac']=(df.plac==1)*1
        
    for nr,datum in enumerate(alla_datum[split_ix:]):
        print(f'{nr+1} av {len(alla_datum[split_ix:])} ',end=': ')

        X_train = df.loc[df.datum<datum,:].copy()
        y_train = X_train.plac; X_train.drop(['plac'],axis=1,inplace=True)

        if classic_test:    ### klassisk train/test utan walkthrough
            X_test  = df.loc[df.datum>=datum,:].copy()
            y_test  = X_test.plac;  X_test.drop(['plac'],axis=1,inplace=True)
            train_pool = Pool(X_train,y_train,cat_features=cat_features)
            test_pool = Pool(X_test,y_test,cat_features=cat_features)
            model.fit(train_pool,use_best_model=True, verbose=verbose,eval_set=test_pool)
        else:
            X_test  = df.loc[df.datum==datum,:].copy()
            y_test  = X_test.plac;  X_test.drop(['plac'],axis=1,inplace=True)
            train_pool = Pool(X_train,y_train,cat_features=cat_features)
            test_pool = Pool(X_test,y_test,cat_features=cat_features)
            model.fit(train_pool,use_best_model=False, verbose=verbose)

        print(model.get_best_iteration(), '\t', round(model.get_best_score()['learn']['Accuracy'],3) )
        ##['validation']['Logloss'],3),'\t', round(model.get_best_score()['validation']['Accuracy:use_weights=true'],3))
        
        if classic_test:    ### klassisk train/test utan walkthrough
            return model,cat_features
        
        model.save_model('model_'+datum)

    X_train =df.copy().drop('plac',axis=1)
    y_train = df.plac 
    model.fit(X_train,y=y_train,cat_features=cat_features)
    print(f'spara model_senaste',datum)
    model.save_model('model_senaste')

    return df,nya_lopp, model,cat_features

In [8]:
#############################################################################################
#  Det här körde jag stegvis när det blev fel och då fungerade det.
#  Därefter fungerade det som det skulle även i orginal.
#  Man kanske skall vänta någon timme efter att alla lopp har körts för att det skall fungera
##############################################################################################

#df, nya_lopp = scrape_nya_lopp()
# nya_lopp.shape
# omg_df = pd.read_csv('..\\omg_att_spela_link.csv')     
# startix=omg_df.Link.str.find('spel')[0]    # index till 'spel' i url
# alla_datum = omg_df.Link.str.slice(start=startix+5,stop=startix+15).to_list()  
# split_ix=0
# print(f'datum att lära från {alla_datum}')

# l2_leaf_regs=2
# model=get_model(use_best=False,iterations=100)
# #   df=remove_features(df.copy())
# cat_features = list(df.loc[:,df.dtypes=='O'].columns)
# df,_ = replace_NaN(df.copy(), cat_features=cat_features)    
# print(f'cat_features {cat_features}\n')

# df['plac']=(df.plac==1)*1

# for nr,datum in enumerate(alla_datum[split_ix:]):
#     print(f'{nr+1} av {len(alla_datum[split_ix:])} ',end=': ')

#     X_train = df.loc[df.datum<datum,:].copy()
#     y_train = X_train.plac; X_train.drop(['plac'],axis=1,inplace=True)

#     if False:    ### klassisk train/test utan walkthrough
#         X_test  = df.loc[df.datum>=datum,:].copy()
#         y_test  = X_test.plac;  X_test.drop(['plac'],axis=1,inplace=True)
#         train_pool = Pool(X_train,y_train,cat_features=cat_features)
#         test_pool = Pool(X_test,y_test,cat_features=cat_features)
#         model.fit(train_pool,use_best_model=True, verbose=True,eval_set=test_pool)
#     else:
#         X_test  = df.loc[df.datum==datum,:].copy()
#         y_test  = X_test.plac;  X_test.drop(['plac'],axis=1,inplace=True)
#         train_pool = Pool(X_train,y_train,cat_features=cat_features)
#         test_pool = Pool(X_test,y_test,cat_features=cat_features)
#         model.fit(train_pool,use_best_model=False, verbose=True)

#     print(model.get_best_iteration(), '\t', round(model.get_best_score()['learn']['Accuracy'],3) )
#     print(model.get_best_iteration(), '\t', round(model.get_best_score()['learn']['Accuracy'],3) )
#     ##['validation']['Logloss'],3),'\t', round(model.get_best_score()['validation']['Accuracy:use_weights=true'],3))
    
#     model.save_model('model_'+datum)

# X_train =df.copy().drop('plac',axis=1)
# y_train = df.plac 
# model.fit(X_train,y=y_train,cat_features=cat_features)
# print(f'spara model_senaste',datum)
# model.save_model('model_senaste')



In [9]:
df, nya_lopp, model, cat_features = walkthrough(classic_test=False, verbose=False)


## Kör allt ovan för walkthrough
### Se till att "..\\omg_att_spela_link.csv" är ifylld

In [10]:
from catboost.utils import eval_metric
df = pd.read_csv('all_data.csv')     
print(df.columns)
dfval=remove_features(df.copy())
cat_features = list(dfval.loc[:,dfval.dtypes=='O'].columns)
dfval,_ = replace_NaN(dfval.copy(), cat_features=cat_features)    
    
validation = nya_lopp.copy()
validation = validation[dfval.columns]
# validation.drop('startnr',axis=1,inplace=True)
# validation = remove_features(validation)
y=validation.plac
y=(y==1)*1
validation.drop('plac',axis=1,inplace=True)
# cat_features = validation.loc[:,validation.dtypes=='O'].columns
        
val_pool=Pool(validation,y,cat_features)
yhat=model.predict(val_pool)
print(utils.get_confusion_matrix(model,val_pool))
eval_metric(yhat,y,'Accuracy')



Index(['datum', 'avd', 'bana', 'häst', 'kusk', 'streck', 'vodds', 'podds',
       'kr', 'spår', 'dist', 'lopp_dist', 'start', 'ålder', 'kön', 'plac',
       'pris', 'h1_dat', 'h1_kusk', 'h1_bana', 'h1_spår', 'h1_plac', 'h1_pris',
       'h1_odds', 'h1_kmtid', 'h2_dat', 'h2_kusk', 'h2_bana', 'h2_spår',
       'h2_plac', 'h2_pris', 'h2_odds', 'h2_kmtid', 'h3_dat', 'h3_kusk',
       'h3_bana', 'h3_spår', 'h3_plac', 'h3_pris', 'h3_odds', 'h3_kmtid',
       'h4_dat', 'h4_kusk', 'h4_bana', 'h4_spår', 'h4_plac', 'h4_pris',
       'h4_odds', 'h4_kmtid', 'h5_dat', 'h5_kusk', 'h5_bana', 'h5_spår',
       'h5_plac', 'h5_pris', 'h5_odds', 'h5_kmtid', 'h1_dist', 'h2_dist',
       'h3_dist', 'h4_dist', 'h5_dist', 'bins', 'h1_auto', 'h2_auto',
       'h3_auto', 'h4_auto', 'h5_auto', 'h1_perf', 'h2_perf', 'h3_perf',
       'h4_perf', 'h5_perf', 'senast', 'delta1', 'delta2', 'delta3', 'delta4',
       'startnr'],
      dtype='object')
[[60. 19.]
 [ 0.  7.]]


[0.7790697674418605]

In [11]:
validation.datum.max()

'2021-08-14'

In [11]:
model.get_feature_importance(prettified=True).head(30)

,Feature Id,Importances
0,streck,64.605765
1,datum,7.857018
2,häst,4.517488
3,h5_kmtid,1.690754
4,h3_odds,1.396984
5,h4_bana,1.200054
6,h5_bana,0.972706
7,kr,0.895608
8,h5_dist,0.853246
9,h3_bana,0.707050
